In [62]:
import time
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras as keras
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models, losses, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import VGG16

In [63]:
data = pd.read_csv('emnist-byclass-mapping.txt', sep=" ", header=None)
data.columns = ['Target','ASCII']

col=[]
for i in data.ASCII.tolist():
    col.append(chr(i))

data['ASCII_Target'] = pd.Series(col)

In [68]:
train_df = pd.read_csv('emnist-byclass-train.csv')
train_df.rename(columns={'35':'Target'}, inplace=True)
train_y = np.array(train_df.pop('Target'))
train_x = train_df

train_x = train_x / 255.0
train_x = train_x.values.reshape([-1, 28, 28, 1])
train_y = keras.utils.to_categorical(train_y)

train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [65]:
test_df = pd.read_csv('emnist-byclass-test.csv')
test_df.rename(columns={'18':'Target'}, inplace=True)

# test_df=pd.merge(test_df, data, how='left', on='Target')
# test_df=test_df.drop(['Target','ASCII'],axis=1)
# test_df.rename(columns={'ASCII_Target':'Target'}, inplace=True)

test_y = np.array(test_df.pop('Target'))
test_x = test_df

test_x = test_x / 255.0
test_x = test_x.values.reshape([-1, 28, 28, 1])
test_y = keras.utils.to_categorical(test_y)

In [66]:
hand_df = pd.read_csv('our_handmade_dataset.csv')
hand_df.rename(columns={'0':'Target'}, inplace=True)

# hand_df=pd.merge(hand_df, data, how='left', on='Target')
# hand_df=hand_df.drop(['Target','ASCII'],axis=1)
# hand_df.rename(columns={'ASCII_Target':'Target'}, inplace=True)

hand_y = np.array(hand_df.pop('Target'))
hand_x = hand_df

hand_x = hand_x / 255.0
hand_x = hand_x.values.reshape([-1, 28, 28, 1])
hand_y = keras.utils.to_categorical(hand_y)

In [84]:
input = Input(shape = (28,28,1))
x = Conv2D (filters =64, kernel_size =3, padding ='same', activation='relu')(input)
x = Conv2D (filters =64, kernel_size =3, padding ='same', activation='relu')(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 2nd Conv Block
x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 3rd Conv block  
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x) 
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x) 
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 4th Conv block
x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 5th Conv block
x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# Fully connected layers  
x = Flatten()(x)
x = Dense(units = 4096, activation ='relu')(x)
x = Dense(units = 4096, activation ='relu')(x)
output = Dense(units = 62, activation ='softmax')(x)
# creating the model

vgg_emnist_model = Model (inputs=input, outputs =output)
vgg_emnist_model.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_22 (InputLayer)       [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_104 (Conv2D)         (None, 28, 28, 64)        640       
                                                                 
 conv2d_105 (Conv2D)         (None, 28, 28, 64)        36928     
                                                                 
 max_pooling2d_40 (MaxPoolin  (None, 14, 14, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_106 (Conv2D)         (None, 14, 14, 128)       73856     
                                                                 
 conv2d_107 (Conv2D)         (None, 14, 14, 128)       147584    
                                                           

In [ ]:
vgg_emnist_model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_vgg_emnist_model.h5", save_best_only=True)
  
vgg_emnist_model.fit(train_x, train_y, epochs=100, validation_data=(val_x, val_y), callbacks=[early_stopping_cb, model_checkpoint_cb])

vgg_emnist_model.evaluate(test_x, test_y)

Epoch 1/100
17449/17449 [==============================] - 8289s 475ms/step - loss: 3.6903 - accuracy: 0.0522 - val_loss: 3.6860 - val_accuracy: 0.0523
Epoch 2/100
 6219/17449 [=========>....................] - ETA: 1:20:35 - loss: 3.6857 - accuracy: 0.0533

In [51]:
my_lenet_5_selu_model.evaluate(hand_x, hand_y)